# Load dataset predictions

To enable tests to make use of predictions, you can load predictions in ValidMind dataset objects in multiple different ways.

This interactive notebook includes the code required to load the demo dataset, preprocess the raw dataset and train a model for testing, and initialize ValidMind objects. Additionally, it offers options for loading predictions using the `assign_predictions()` function, such as loading predictions from a file, linking an existing prediction column in the dataset with a model, or allowing the developer framework to run and link predictions to a model.

## Contents
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Initialize the client library](#toc3_)    
  - [Preview the documentation template](#toc3_1_)    
- [Load the sample dataset](#toc4_)    
- [Prepocess the raw dataset](#toc5_)    
- [Train models for testing](#toc6_)    
- [Initialize ValidMind objects](#toc7_)    
  - [Initialize the ValidMind models](#toc7_1_)    
  - [Initialize the ValidMind datasets](#toc7_2_)    
- [Options to load predictions using the developer frameworks](#toc8_)    
  - [Load predictions from a file](#toc8_1_)    
  - [Predictions calculated outside of VM](#toc8_2_)    
  - [Assign predictions to the training dataset](#toc8_3_)    
  - [Run an example test](#toc8_4_)    
  - [Link an existing prediction column in the dataset with a model](#toc8_5_)    
    - [Link prediction column to a specific model](#toc8_5_1_)    
  - [Link an existing prediction column in the dataset with a model](#toc8_6_)    
    - [Pass `<vm_model>` in dataset interface](#toc8_6_1_)    
    - [Through `assign_predictions` interface](#toc8_6_2_)    
  - [Run an example test](#toc8_7_)    
- [Next steps](#toc9_)    
  - [Work with your model documentation](#toc9_1_)    
  - [Discover more learning resources](#toc9_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## About ValidMind

ValidMind is a platform for managing model risk, including risk associated with AI and statistical models. You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?

If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

::: {.callout-tip}

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)

:::

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.


<a id='toc2_'></a>

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Binary classification`
   - Use case: `Marketing/Sales - Attrition/Churn Management`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

<a id='toc3_1_'></a>

### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:


In [ ]:
vm.preview_template()

<a id='toc4_'></a>

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


In [ ]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()
raw_df.head()

<a id='toc5_'></a>

## Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

<a id='toc6_'></a>

## Train models for testing

- Initialize XGBoost and Logistic Regression Classifiers


In [ ]:
from sklearn.linear_model import LogisticRegression
import xgboost

%matplotlib inline

xgb = xgboost.XGBClassifier(early_stopping_rounds=10)
xgb.set_params(
    eval_metric=["error", "logloss", "auc"],
)
xgb.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

lr = LogisticRegression(random_state=0)
lr.fit(
    x_train,
    y_train,
)


<a id='toc7_'></a>

## Initialize ValidMind objects

<a id='toc7_1_'></a>

### Initialize the ValidMind models


In [ ]:
vm_model_xgb = vm.init_model(
    xgb,
    input_id="xgb",
)
vm_model_lr = vm.init_model(
    lr,
    input_id="lr",
)

<a id='toc7_2_'></a>

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset
- `class_labels` — an optional value to map predicted classes to class labels

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [ ]:
vm_raw_ds = vm.init_dataset(
    input_id="raw_dataset",
    dataset=raw_df,
    target_column=demo_dataset.target_column,
)

vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    target_column=demo_dataset.target_column,
)
vm_test_ds = vm.init_dataset(
    input_id="test_dataset", dataset=test_df, target_column=demo_dataset.target_column
)

<a id='toc8_'></a>

## Options to load predictions using the developer frameworks

<a id='toc8_1_'></a>

### Load predictions from a file

This creates a new column called `<model_id>_prediction` in the dataset and assigns metadata to track that the `<model_id>_prediction` column is linked to the model `<model_id>`


<a id='toc8_2_'></a>

### Predictions calculated outside of VM


In [ ]:
import pandas as pd

train_xgb_prediction = pd.DataFrame(xgb.predict(x_train), columns=["xgb_prediction"])
test__xgb_prediction = pd.DataFrame(xgb.predict(x_val), columns=["xgb_prediction"])

train_lr_prediction = pd.DataFrame(lr.predict(x_train), columns=["lr_prediction"])
test_lr_prediction = pd.DataFrame(lr.predict(x_val), columns=["lr_prediction"])

<a id='toc8_3_'></a>

### Assign predictions to the training dataset

We can now use the `assign_predictions()` method from the `Dataset` object to link existing predictions to any model:


In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model_xgb, prediction_values=train_xgb_prediction.xgb_prediction.values
)
vm_train_ds.assign_predictions(
    model=vm_model_lr, prediction_values=train_lr_prediction.lr_prediction.values
)

<a id='toc8_4_'></a>

### Run an example test

Now, let's run an example test such as `MinimumAccuracy` twice to show how we're able to load the correct model predictions by using the `model` input parameter, even though we're passing the same `train_ds` dataset instance to the test:


In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={"dataset": vm_train_ds, "model": vm_model_xgb},
)

In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={
        "dataset": vm_train_ds,
        "model": vm_model_lr,
    },
)

<a id='toc8_5_'></a>

### Link an existing prediction column in the dataset with a model

This approach allows loading datasets that already have prediction columns in addition to feature and target columns. The developer framework assigns metadata to track the predictions column that are linked to a given `<vm_model>` model.


In [ ]:
train_df2 = train_df.copy()
train_df2["xgb_prediction"] = train_xgb_prediction.xgb_prediction.values
train_df2["lr_prediction"] = train_lr_prediction.lr_prediction.values
train_df2.head(5)

In [ ]:
feature_columns = [
    "CreditScore",
    "Gender",
    "Age",
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary",
    "Geography_France",
    "Geography_Germany",
    "Geography_Spain",
]

vm_train_ds = vm.init_dataset(
    dataset=train_df2,
    input_id="train_dataset",
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
)

<a id='toc8_5_1_'></a>

#### Link prediction column to a specific model

The `prediction_column` parameter informs the `Dataset` object about the model that should be linked to that column.


In [ ]:
vm_train_ds.assign_predictions(model=vm_model_xgb, prediction_column="xgb_prediction")
vm_train_ds.assign_predictions(model=vm_model_lr, prediction_column="lr_prediction")

In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={"dataset": vm_train_ds, "model": vm_model_xgb},
)

In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={"dataset": vm_train_ds, "model": vm_model_lr},
)

<a id='toc8_6_'></a>

### Link an existing prediction column in the dataset with a model

This lets the developer framework run model predictions, creates a new column called `<model_id>_prediction`, and assign metadata to track that the `<model_id>_prediction` column is linked to the `<vm_model>` model.

There are two ways run and assign model predictions with the developer framework:

- When initializing a `Dataset` with `init_dataset()`. This is the most straightforward method to assign predictions for a single model.
- Using `dataset.assign_predictions()`. This allows assigning predictions to a dataset for one or more models.


<a id='toc8_6_1_'></a>

#### Pass `<vm_model>` in dataset interface


In [ ]:
feature_columns = [
    "CreditScore",
    "Gender",
    "Age",
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary",
    "Geography_France",
    "Geography_Germany",
    "Geography_Spain",
]

vm_train_ds = vm.init_dataset(
    model=vm_model_xgb,
    dataset=train_df,
    input_id="train_dataset",
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
)

<a id='toc8_6_2_'></a>

#### Through `assign_predictions` interface


In [ ]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
)

##### Perform predictions using the same `assign_predictions` interface


In [ ]:
vm_train_ds.assign_predictions(model=vm_model_xgb)
vm_train_ds.assign_predictions(model=vm_model_lr)

<a id='toc8_7_'></a>

### Run an example test

Now, let's run an example test such as `MinimumAccuracy` twice to show how we're able to load the correct model predictions by using the `model` input parameter, even though we're passing the same `train_ds` dataset instance to the test:


In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={"dataset": vm_train_ds, "model": vm_model_xgb},
)

In [ ]:
full_suite = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={
        "dataset": vm_train_ds,
        "model": vm_model_lr,
    },
)

<a id='toc9_'></a>

## Next steps

You can look at the output produced by the ValidMind Developer Framework right in the notebook where you ran the code, as you would expect. But there is a better way: use the ValidMind platform to work with your model documentation.

<a id='toc9_1_'></a>

### Work with your model documentation

1. From the [**Model Inventory**](https://app.prod.validmind.ai/model-inventory) in the ValidMind Platform UI, go to the model you registered earlier.

2. Click and expand the **Model Development** section.

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/working-with-model-documentation.html)

<a id='toc9_2_'></a>

### Discover more learning resources

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.